###### PRECINCT NAME MATCHING
Takes multiple datasets with `county` and `precinct_name` columns and builds a cross-dataset reference table of precinct name variations.

In [6]:
import os
import re
import jaro
import config
import folium
import operator
import numpy as np
import pandas as pd
import geopandas as gp
from fuzzywuzzy import fuzz
from IPython import display
from tqdm.notebook import tqdm

config.data_dict = {}
config.county_dict = {}
d_types = ['SHP16', 'NOV18', 'SHP18', 'NOV20', 'SHP20'] # add more data here.

### To add more data:
- Invent a string label for the new dataset and add it to `d_types` (above)
- Create a new k:v pair in config.data_dict with data details (key should be the pre-selected d_type label)
    - Follow the format of the other datasets in section **1b**
    - Identify the name of the column describing County, set that as `county_col`
    - Identify the names of the columns describing precincts (usually multiple), set the list as `precinct_cols`
- Run the entire notebook
- Use the verification methods at the end of the notebook to ID errors
    - Use `replace_strs` to direct-edit specific typos 
    - Use `county_edit` to apply a cleaning function to precinct names in a specific county for a given dataset
        - `source_col` is the column to apply to function to
        - `target_col` is the column where the cleaned values will be written to
            - Can be a new feature, or can be an existing feature (existing means overwriting original values)

# 1. Data

### 1a. Cleaning functions

In [7]:
# for NOV18
def convert_rockdale(ab):
    if str(ab).lower()=='nan': return ab
    rockdale_pmap = {'BA': 'Barkside',
                     'BT': 'Bethel',
                     'CO': 'Conyers',
                     'FI': 'Fieldstone',
                     'FS': 'Flat Shoals',
                     'HC': 'Honey Creek',
                     'HI': 'High Tower',
                     'LA': 'The Lakes',
                     'LO': 'Lorraine',
                     'MA': 'Magnet',
                     'MI': 'Milestead',
                     'OT': 'Olde Town',
                     'RO': 'Rockdale',
                     'SM': 'Smyrna',
                     'SP': 'St. Pius',
                     'ST': 'Stanton'}
    ab = ab.upper()
    if ab in rockdale_pmap.keys():
        return rockdale_pmap[ab]
    else: return ab

In [8]:
# FOR SHP18
def convert_spalding(n):
    if str(n).lower()=='nan': return n
    if len(n)<2: return'0'+n
    else: return n

In [9]:
# FOR NOV20
def get_barrow_num(x):
    if str(x).lower()=='nan': return x
    return x.split()[0]

def get_barrow_name(x):
    if str(x).lower()=='nan': return x
    return ' '.join(re.findall('[A-Za-z]*', x))

---

### 1b. Define data params

In [10]:
config.data_dict['SHP16'] = {
    'county_col': 'CTYNAME',
    'precinct_cols': ['PRECINCT_I', 'PRECINCT_N'],
    # although there are multiple version of this dataset, we assume precinct names remain consistent within
    'fpath': '../../shapes/precincts_2016/GA_precincts16.shp',
    'replace_strs' : { },
    'county_edits': [ ]
}

In [17]:
config.data_dict['NOV18'] = {
    'county_col': 'County',
    'precinct_cols': ['PRECINCT ID', 'PRECINCT DESCRIPTION'],
    # although there are multiple version of this dataset, we assume precinct names remain consistent within
    'fpath': '../2018_november/participation_demography/by_precinct/democrats.csv',
    'replace_strs' : {
        'austin \(dun\)': 'austin', # need to escape ()'s for .str.replace in pandas
        'avondale \(avo\)': 'avondale',
        'lithonia \(lit\)': 'lithonia',
        'woodward \(bhavn\)': 'woodward',
        'fbc - flc': 'family life center',
    },
    'county_edits': [{
        'county': 'rockdale',
        'target_col': 'PRECINCT DESCRIPTION',
        'convert_func': convert_rockdale, # this function is defined above!
    }]
}

In [18]:
config.data_dict['SHP18'] = {
    'county_col': 'locality',
    'precinct_cols': ['prec_shp', 'prec_elec'],
    'fpath': '../../shapes/precincts_2018/2018Precincts.shp',
    'replace_strs' : {
        'hoggard mill': 'hoggards mill',
        'south mill': 'south milledgeville',
        'north mill': 'north milledgeville',
        'bethlehem church - 211': 'bethlehem church',
        'chattahoochee acvitity center': 'activity center' ,
        'cjc': '#3 cjc',
    },
    'county_edits': [{
        'county': 'spalding',
        'target_col': 'prec_shp',
        'convert_func': convert_spalding,
    },{
        'county': 'barrow',
        'source_col': 'prec_elec',
        'target_col': 'prec_elec',
        'convert_func': get_barrow_num,
    },]   
}

In [19]:
config.data_dict['SHP20'] = {
    'county_col': 'CTYNAME',
    'precinct_cols': ['PRECINCT_I', 'PRECINCT_N'],
    'fpath': '../../shapes/precincts_2020/ga_2020_general.shp',
    'replace_strs' : {
        'hoggard mill': 'hoggards mill',
        'south mill': 'south milledgeville',
        'north mill': 'north milledgeville',
        'bethlehem church - 211': 'bethlehem church',
        'chattahoochee acvitity center': 'activity center' ,
        'cjc': '#3 cjc',
    },
    'county_edits': False, 
}

In [20]:
config.data_dict['NOV20'] = {
    'county_col': 'County',
    'precinct_cols': ['Precinct'],
    'fpath': '../2020_november/candidate_votes/by_precinct/US Senate (Loeffler) - Special.csv',
    'replace_strs' : {
        '03 Hmong New Hope Alliance Church 6364':'bramlett elementary',
        '04 Covenant Life Sanctuary': 'westside middle',
    },
    'county_edits': [{
        'county': 'barrow',
        'source_col': 'Precinct',
        'target_col': 'Precinct ID',
        'convert_func': get_barrow_num,
    },
       {'county': 'barrow',
        'target_col': 'Precinct',
        'convert_func': get_barrow_name,
    }]  
}

### data import function

In [21]:
def read_data(data_key):    
    print(f'> Reading {data_key}...')
    data = config.data_dict[data_key].copy()
    
    # read data
    if 'SHP' in data_key: df = gp.read_file(data['fpath'])
    else: df = pd.read_csv(data['fpath'])

    # isolate & clean column names
    df = df[ [data['county_col']] + data['precinct_cols'] ]
    df = df.rename(columns={data['county_col']: 'county'})
    df.dropna(subset=['county'], inplace=True)
    
    # create "_CLEAN" lowercase version columns
    cols = df.columns
    for c in cols:
        df[c+'_CLEAN'] = df[c].str.lower().copy()
       
    # fix specific typos
    typos_fixed = 0
    for o, n in data['replace_strs'].items():
        for c in cols: # assigns edits to CLEAN version of column
            fixed = df[c+'_CLEAN'].str.replace( o.lower(), n.lower() ).copy()
            # calc how many values were affected by the edit (for log)
            typos_fixed += (fixed.dropna().values != df[c+'_CLEAN'].dropna().values).sum() 
            df[c+'_CLEAN'] = fixed # assign edits
    print(f"  > {typos_fixed}/{len(data['replace_strs'])} typos fixed")
    
    # county-specific edit functions
    # given_function('source_col')——>'target_col'
    c_edits = config.data_dict[data_key]['county_edits']
    if c_edits:
        for ce in c_edits:
            c_idx = df[df.county.str.lower()==ce['county'].lower()].index
            # create new column if target doesn't exist
            if 'source_col' not in ce.keys():
                ce['source_col'] = ce['target_col']            
            # assign the pre-made function:
            df.loc[c_idx, ce['target_col']+'_CLEAN'
                  ] = df.loc[c_idx, ce['source_col']
                  ].apply(ce['convert_func'])
            
    # reset index
    df = df.reset_index(drop=True) # ******** MAYBE DON'T DO THIS
    
    # save cleaned df to data_dict
    config.data_dict[data_key]['data'] = df

    # re-iterate over found data for all counties to build county_dict
    all_counties = list(df.county_CLEAN.unique())
    for county in all_counties:
        if county not in config.county_dict.keys():
            config.county_dict[county] = {}
        c_data = df[df.county_CLEAN.str.lower()==county.lower()]
        c_data = c_data[[c for c in c_data.columns
                         if 'county' not in c]] # drop county col
        config.county_dict[county][d_key  # store county data in county_dict
                                  ] = c_data.reset_index(drop=True)
    
    # log
    n_precs = max([df[c+'_CLEAN'].nunique() for c in data['precinct_cols']])
    print('  >', len(all_counties), 'counties')
    print('  >', n_precs, 'precincts')
    print()

---

## 1c. Load data

In [22]:
for d_key in d_types:
    read_data(d_key)

> Reading SHP16...
  > 0/0 typos fixed
  > 159 counties
  > 2555 precincts

> Reading NOV18...
  > 5/5 typos fixed
  > 159 counties
  > 2533 precincts

> Reading SHP18...
  > 12/6 typos fixed
  > 159 counties
  > 2568 precincts

> Reading NOV20...
  > 1/2 typos fixed
  > 159 counties
  > 2556 precincts

> Reading SHP20...
  > 4/6 typos fixed
  > 159 counties
  > 2573 precincts



---

# 3. Find matches across all datasets in one county

In [23]:
# order datasets by top number of precincts (to sort the joining process)
def sort_datasets(county):

    p_counts = {} # to store n_precincts per dataset (in this county)
    c_dict = config.county_dict[county] # load existing data for this county
    for add_d_key in list(c_dict.keys()): # datasets available for this county

        data = config.data_dict[add_d_key].copy() # metadata
        add_data = c_dict[add_d_key].copy() #df

        # don't count absentee columns
        n_precincts = max(
            [add_data[p_col].nunique()
             for p_col in data['precinct_cols']]) 
        p_counts[add_d_key] = n_precincts

    # sorted datasets by top n_precincts (in this county)
    top_d_types = {k: v for k, v in
                    sorted(p_counts.items(),
                    key=lambda item: item[1])[::-1]}
    return top_d_types

In [24]:
# build joined reference table, starting with the dataset with the most precincts 
def build_aka_table(county, verb=False):

    top_d_types = sort_datasets(county) # get dataset add order
    
    for i, d_add_key in enumerate(top_d_types): # iterate datasets 

        # target existing data
        c_dict = config.county_dict[county]
        df = c_dict[d_add_key].copy()
        # add datatype specifier to column names
        df.rename(columns={
            c:d_add_key+'_'+c for c in df.columns}, inplace=True)
        
        
        # find or create new 'joined' table for this county
        if 'joined' not in c_dict.keys(): 
            # if new, instantiate "joined" as the first (biggest) existing data
            config.county_dict[county]['joined'] = df
            continue # done with this dataset
        # load existing joined data
        else: pre_joined = config.county_dict[county]['joined'].copy()

        # identify columns to compare over
        pre_cols = [c for c in pre_joined.columns if 'CLEAN' in c]
        new_cols = [c for c in df.columns if 'CLEAN' in c]

        
        # SEARCH FOR MATCHES...
        match_hist_idx = {}

        # for each new row ...
        for n_i in df.index:
            best_score = .75
            perfect_find = False
            new_row = df.loc[n_i]

            # for each val (comp col) in new row...
            for new_ccol in new_cols:
                if perfect_find: break
                n_val = new_row[new_ccol]
                if str(n_val).lower() in ['nan', '88888', '99999']: continue

                # for each existing feature name... (comp col)
                for pre_ccol in pre_cols:
                    if perfect_find: break
                    pre_series = pre_joined[pre_ccol].dropna() 

                    # for each val in comp col
                    for p_i, p_val in dict(pre_series).items():
                        if perfect_find: break
                        if str(n_val).lower() in ['nan', '88888', '99999']: continue

                        # PERFECT MATCH    
                        if n_val==p_val:
                            match_hist_idx[n_i] = p_i
                            perfect_find = True

                        # JARO MATCH
                        else:
                            j_score = jaro.jaro_metric(n_val.upper(), p_val.upper())
                            if j_score > best_score:
                                best_score = j_score
                                match_hist_idx[n_i] = p_i

                                
        # DATA MATCH LOG
        could_match = df.copy()
        for c in new_cols:
            could_match = could_match[(could_match[c]!='99999') &
                                      (could_match[c]!='88888')]
        #could_match = could_match.dropna()
        
        pct_found = round(100*(len(match_hist_idx)/len(could_match)), 1)
        if verb and pct_found<90:
            print(f'{county.upper()} ——— {d_add_key} >>> M: {pct_found}%')
        
        # COUNTY LOG
        config.county_dict[county][d_add_key+'_SEARCHED'] = len(df)
        config.county_dict[county][d_add_key+'_FOUND'] = len(match_hist_idx)
        config.county_dict[county][d_add_key+'_MRATE'] = pct_found
        
        # all comparisons done for this dataset....
        # ASSIGN MATCHES (index vals) to 'joined' dataframe
        pre_joined['match_idx'] = np.nan # empty init
        for n_i in df.index:
            if n_i in match_hist_idx.keys(): # if found, assign index of match
                pre_joined.loc[match_hist_idx[n_i], 'match_idx'] = n_i              

        # merge & updated county 'join' table 
        new_joined = pd.merge(pre_joined, df, how='outer',
                              left_on='match_idx', right_index=True)
        new_joined.drop('match_idx', axis=1, inplace=True)
        new_joined.reset_index(drop=True, inplace=True)

        # write new merge back over county_dict
        config.county_dict[county]['joined'] = new_joined.copy()
        


---

# 4. Iterate all counties to find matches

In [25]:
counties = config.county_dict.keys()
for county in tqdm(counties):
    build_aka_table(county, True)

SPALDING ——— NOV20 >>> M: 77.8%
BUTTS ——— NOV18 >>> M: 0.0%
LAURENS ——— SHP16 >>> M: 88.2%
JACKSON ——— NOV18 >>> M: 0.0%
CHATTAHOOCHEE ——— SHP18 >>> M: 50.0%



---

# 5. Review matches

In [26]:
def print_match_rate(d_add_key):
    total_searched = 0
    total_found = 0
    for v in config.county_dict.values():
        try: total_searched += v[d_add_key+'_SEARCHED']
        except: pass
        try: total_found += v[d_add_key+'_FOUND']
        except: pass

    pct_found = (round(total_found / total_searched, 3)*100)
    
    print(f"{d_add_key}:  {pct_found}% found")
    
for d in d_types:
    print_match_rate(d)

SHP16:  99.8% found
NOV18:  96.8% found
SHP18:  99.8% found
NOV20:  99.7% found
SHP20:  99.9% found


***Assess addtl required edits here & add `county_edits` or `replace_strings` to the problem data as needed.***

---

# 6. Merge all counties

In [27]:
out = pd.DataFrame()

for county, data in config.county_dict.items():

    county_precs = data['joined']
    county_precs['county'] = county
    out = pd.concat([out, data['joined']])
    
out.reset_index(drop=True, inplace=True)

# 7. Infer precinct names & codes

In [28]:
# work with clean cols only
o_clean = out[[c for c in out.columns if 'CLEAN' in c]].copy()

for r_idx in o_clean.index:
    
    r_vals = list(o_clean.loc[r_idx].dropna().values)
    r_vals = set([c.upper() for c in r_vals]) 
    r_vals = {r: len(r) for r in r_vals}
    
    if r_vals:
        longest = max(r_vals.items(), key=operator.itemgetter(1))[0]
        shortest = min(r_vals.items(), key=operator.itemgetter(1))[0]

        if shortest in longest.split():
            longest = ' '.join(longest.split(shortest))
        
        if shortest == longest:
            nums_in_long = len([c for c in re.findall('[\d]*', longest) if c!=''])
            if len(longest) <= 8 and nums_in_long:
                longest = ''
            else: shortest = ''

        shortest = shortest.upper()
        
        l_parts = longest.split('(')
        
        longest = ' '.join([l.capitalize() for l in l_parts[0].split()])
        if len(l_parts)>1:
            longest +=' ('+l_parts[1].lower()
                
        out.loc[r_idx, 'precinct_name'] = longest
        out.loc[r_idx, 'precinct_id'] = shortest

out = out[list(out.columns)[-3:] + list(out.columns)[:-3]]
out.county = out.county.apply(lambda x: ' '.join([p.capitalize() for p in x.split()]))
out.sort_values('precinct_name').sort_values('county')

,county,precinct_name,precinct_id,SHP20_PRECINCT_I,SHP20_PRECINCT_N,SHP20_PRECINCT_I_CLEAN,SHP20_PRECINCT_N_CLEAN,NOV20_Precinct,NOV20_Precinct_CLEAN,NOV20_Precinct ID_CLEAN,...,NOV18_PRECINCT ID_CLEAN,NOV18_PRECINCT DESCRIPTION_CLEAN,SHP18_prec_shp,SHP18_prec_elec,SHP18_prec_shp_CLEAN,SHP18_prec_elec_CLEAN,SHP16_PRECINCT_I,SHP16_PRECINCT_N,SHP16_PRECINCT_I_CLEAN,SHP16_PRECINCT_N_CLEAN
2041,Appling,,3C,3C,3C,3c,3c,3C,3c,NaN,...,3c,3c,3c,3C,3c,3c,3C,3C,3c,3c
2029,Appling,,4B,4B,4B,4b,4b,4B,4b,NaN,...,4b,4b,4b,4B,4b,4b,4B,4B,4b,4b
2031,Appling,,4A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4A,4A,4a,4a
2030,Appling,,3B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3B,3B,3b,3b
2034,Appling,,5C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5C,5C,5c,5c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,Worth,Sumner,4,4,SUMNER,4,sumner,Sumner,sumner,NaN,...,4,sumner,Sumner,Sumner,sumner,sumner,4,SUMNER,4,sumner
1724,Worth,Red Rock,6,6,RED ROCK,6,red rock,Red Rock,red rock,NaN,...,6,red rock,Red Rock,Red Rock,red rock,red rock,6,RED ROCK,6,red rock
1735,Worth,Bridgeboro,12,12,BRIDGEBORO,12,bridgeboro,Bridgeboro,bridgeboro,NaN,...,12,bridgeboro,Bridgeboro,Bridgeboro,bridgeboro,bridgeboro,12,BRIDGEBORO,12,bridgeboro
1731,Worth,Poulan,3,3,POULAN,3,poulan,Poulan,poulan,NaN,...,3,poulan,Poulan,Poulan,poulan,poulan,3,POULAN,3,poulan


In [30]:
out.to_csv('precinct_nametable.csv', index=False)